# Engine

This document describes the performance evaluation for [Typesense](https://typesense.org/).

For the sake of testing a local docker single-instance cluster was used via the following `docker-compose.yml`:

```yaml
version: '3.4'
services:
  typesense:
    image: typesense/typesense:0.24.0
    restart: on-failure
    ports:
      - "8108:8108"
    volumes:
      - ./typesense-data:/data
    command: '--data-dir /data --api-key=xyz --enable-cors'
```

The data used to test the engine comes from [the Lichess database](https://database.lichess.org) and represents chess games.
The full dataset can be downloaded using the following script:

```shell
#!/bin/bash

rm -rf datasets
mkdir datasets
cd datasets || exit

for i in $(seq -w 1 12); do
  curl https://database.lichess.org/standard/lichess_db_standard_rated_2013-"$i".pgn.zst -o lichess_db_standard_rated_2013-"$i".pgn.zst
  pzstd -d lichess_db_standard_rated_2013-"$i".pgn.zst
  rm lichess_db_standard_rated_2013-"$i".pgn.zst
done

date > downloaded_at.txt
```

For the sake of performance evaluation, a small set of 100 000 chess games was taken from the beginning of [the january 2013 standard games dataset](https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst).

# Interfacing with the engine

Out of the box typesense does not provide any graphical user interface.
While there are client libraries provided by first party vendor as well as the community, they are simply thin wrappers over the REST HTTP API.

Quoting the website itself:
>Typesense has a RESTful API so it can be used with any HTTP library in any programming language, even the ones not listed below. The client libraries are just thin wrappers around Typesense's HTTP API.

# Metrics

The following code shows extraction of performance metrics from the server instance.

In [52]:
typesense_url = "http://localhost:8108"
typesense_api_key = "xyz"
typesense_api_key_header = "X-TYPESENSE-API-KEY"
batch_size = 100
batch_count = 100

In [53]:
import requests

measurements = []

# noinspection PyShadowingBuiltins
def human_size(bytes, units=None):
    if units is None:
        units = [' bytes', 'KB', 'MB', 'GB', 'TB', 'PB', 'EB']
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes >> 10, units[1:])


def measure_metrics():
    measurement = requests.get(f"{typesense_url}/metrics.json",
                               headers={typesense_api_key_header: typesense_api_key}).json()
    measurements.append(measurement)
    return measurement


def display_metrics_text(measurement):
    print(f"CPU:                                  {measurement['system_cpu_active_percentage']}%")
    print(f"system_disk_used_bytes:               {human_size(int(measurement['system_disk_used_bytes']))}")
    print(f"system_memory_used_bytes:             {human_size(int(measurement['system_memory_used_bytes']))}")
    print(f"system_network_received_bytes:        {human_size(int(measurement['system_network_received_bytes']))}")
    print(f"system_network_sent_bytes:            {human_size(int(measurement['system_network_sent_bytes']))}")
    print(f"typesense_memory_active_bytes:        {human_size(int(measurement['typesense_memory_active_bytes']))}")
    print(f"typesense_memory_allocated_bytes:     {human_size(int(measurement['typesense_memory_allocated_bytes']))}")
    print(f"typesense_memory_mapped_bytes:        {human_size(int(measurement['typesense_memory_mapped_bytes']))}")
    print(f"typesense_memory_metadata_bytes:      {human_size(int(measurement['typesense_memory_metadata_bytes']))}")
    print(f"typesense_memory_fragmentation_ratio: {measurement['typesense_memory_fragmentation_ratio']}")
    print(f"typesense_memory_resident_bytes:      {human_size(int(measurement['typesense_memory_resident_bytes']))}")
    print(f"typesense_memory_retained_bytes:      {human_size(int(measurement['typesense_memory_retained_bytes']))}")

Create empty collection.

In [54]:
import chess.pgn
from datetime import datetime
import typesense
from timeit import default_timer as timer

typesense_client = typesense.Client({
    'nodes': [{
        'host': 'localhost',
        'port': '8108',
        'protocol': 'http'
    }],
    'api_key': 'xyz',
    'connection_timeout_seconds': 2
})

collection_name = 'chess'

schema = {
    'name': collection_name,
    'fields': [
        {'name': 'link', 'type': 'string'},  # [Site "https://lichess.org/j1dkb5dw"]
        {'name': 'timestamp_utc', 'type': 'int32'},  # [UTCDate "2012.12.31"] [UTCTime "23:04:12"]
        {'name': 'event', 'type': 'string'},  # [Event "Rated Classical game"]
        {'name': 'white', 'type': 'string'},  # [White "BFG9k"]
        {'name': 'black', 'type': 'string'},  # [Black "mamalak"]
        {'name': 'opening', 'type': 'string'},  # [Opening "French Defense: Normal Variation"]
        {'name': 'termination', 'type': 'string'},  # [Termination "Normal"]
        {'name': 'mainline_moves', 'type': 'string'},  # 1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6 gxh6 6. Be2...
        # {'name': 'sequenced_moves', 'type': 'string'},  # e4 e6 d4 b6 a3 Bb7 Nc3 Nh6 Bxh6 gxh6 Be2...
    ],
    'default_sorting_field': 'timestamp_utc'
}

names = [x['name'] for x in typesense_client.collections.retrieve()]

if collection_name in names:
    typesense_client.collections[collection_name].delete()

print("before schema created")
display_metrics_text(measure_metrics())
print()
typesense_client.collections.create(schema)
print("after schema created")
display_metrics_text(measure_metrics())

before schema created
CPU:                                  0.00%
system_disk_used_bytes:               147GB
system_memory_used_bytes:             1GB
system_network_received_bytes:        18MB
system_network_sent_bytes:            15MB
typesense_memory_active_bytes:        71MB
typesense_memory_allocated_bytes:     57MB
typesense_memory_mapped_bytes:        124MB
typesense_memory_metadata_bytes:      13MB
typesense_memory_fragmentation_ratio: 0.20
typesense_memory_resident_bytes:      71MB
typesense_memory_retained_bytes:      208MB

after schema created
CPU:                                  2.38%
system_disk_used_bytes:               147GB
system_memory_used_bytes:             1GB
system_network_received_bytes:        18MB
system_network_sent_bytes:            15MB
typesense_memory_active_bytes:        71MB
typesense_memory_allocated_bytes:     57MB
typesense_memory_mapped_bytes:        124MB
typesense_memory_metadata_bytes:      13MB
typesense_memory_fragmentation_ratio: 0.20
types

Fill collection with data.

In [ ]:
collection = typesense_client.collections[collection_name]

durations = []
with open('datasets/lichess_db_standard_rated_2013-01.pgn') as file:
    counter = 0
    while True:
        game = chess.pgn.read_game(file)
        if game is None:
            break
        counter = counter + 1
        if counter > batch_size * batch_count:
            break

        date = game.headers.get('UTCDate')  # 2012.12.31
        time = game.headers.get('UTCTime')  # 23:04:12

        ts_game = {
            'id': game.headers.get('Site').split('/')[-1],  # [Site "https://lichess.org/j1dkb5dw"]
            'link': game.headers.get('Site'),
            'timestamp_utc': int(datetime.strptime(f'{date} {time}', '%Y.%m.%d %H:%M:%S').timestamp()),
            'event': game.headers.get('Event'),
            'white': game.headers.get('White'),
            'black': game.headers.get('Black'),
            'opening': game.headers.get('Opening'),
            'termination': game.headers.get('Termination'),
            'mainline_moves': str(game.mainline_moves()),
        }

        start_timer = timer()
        collection.documents.upsert(ts_game)
        stop_timer = timer()
        duration = stop_timer - start_timer
        durations.append(duration)

        if counter % batch_size == 0:
            measure_metrics()
            print(f'Upsert batch {int(counter/batch_size)} in {sum(durations[-batch_size:]) * 1000:.2f}ms')

Upsert batch 1 in 481.89ms
Upsert batch 2 in 350.15ms
Upsert batch 3 in 360.65ms
Upsert batch 4 in 318.41ms
Upsert batch 5 in 346.19ms
Upsert batch 6 in 321.45ms
Upsert batch 7 in 331.04ms
Upsert batch 8 in 352.88ms
Upsert batch 9 in 333.81ms
Upsert batch 10 in 363.73ms
Upsert batch 11 in 375.28ms
Upsert batch 12 in 352.07ms
Upsert batch 13 in 387.83ms
Upsert batch 14 in 436.27ms
Upsert batch 15 in 406.88ms
Upsert batch 16 in 513.42ms
Upsert batch 17 in 417.83ms
Upsert batch 18 in 404.42ms
Upsert batch 19 in 362.19ms
Upsert batch 20 in 398.37ms
Upsert batch 21 in 373.33ms
Upsert batch 22 in 391.43ms
Upsert batch 23 in 450.47ms
Upsert batch 24 in 369.11ms
Upsert batch 25 in 311.72ms
Upsert batch 26 in 343.33ms
Upsert batch 27 in 317.96ms
Upsert batch 28 in 347.87ms
Upsert batch 29 in 348.07ms
Upsert batch 30 in 319.03ms
Upsert batch 31 in 349.11ms
Upsert batch 32 in 489.38ms
Upsert batch 33 in 391.23ms
Upsert batch 34 in 398.42ms
Upsert batch 35 in 389.71ms
Upsert batch 36 in 339.42ms
U

Analyze performance.

In [ ]:
import matplotlib.pyplot as plt

def plot(data, name):
    x = [i for i, _ in enumerate(data)]
    y = data

    plt.xlabel('measurement [every 10 games]')
    plt.ylabel(name)
    plt.scatter(x, y)
    plt.show()

def plot_metric(metric, name, transform=lambda x: x):
    plot([transform(measurement[metric]) for measurement in measurements], name)


plot_metric("system_disk_used_bytes", "DISK MB", lambda x: float(x)/1024.0/1024.0)
plot_metric("system_cpu_active_percentage", "CPU %", lambda x: float(x))
plot_metric("system_memory_used_bytes", "MEMORY MB", lambda x: float(x)/1024.0/1024.0)
plot(durations, "UPSERT DURATION ms")